# 基于逻辑回归的信用欺诈预测识别模型实验
本实验将演示如何使用逻辑回归模型进行信用卡欺诈识别，包括数据读取、分析、预处理、建模、评估及结论。

## 1. 读取数据
数据集文件名为 `creditcard.csv`，位于当前目录。

In [ ]:
import pandas as pd
df = pd.read_csv('creditcard.csv')
# 查看数据维度
df.shape

## 2. 描述性分析与可视化分析

In [ ]:
# 查看数据的随机五项
df.sample(5)

In [ ]:
# 查看缺失值
df.isnull().sum().sum()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.countplot(x='Class', data=df)
plt.show()

In [ ]:
df.hist(figsize=(24,16))
plt.show()

## 3. 数据预处理

In [ ]:
# 对Amount进行标准化处理，存储在normal_amount字段中
from sklearn.preprocessing import StandardScaler
df['normal_amount'] = StandardScaler().fit_transform(df['Amount'].values.reshape(-1,1))
# 去掉Amount和Time字段
df = df.drop(['Amount','Time'], axis=1)
df.columns

In [ ]:
# 划分自变量和因变量
X = df.loc[:,df.columns != 'Class']
y = df.loc[:,df.columns == 'Class']
# 划分训练集和测试集
from sklearn.model_selection import train_test_split
X_tr,X_ts,y_tr,y_ts = train_test_split(X,y,test_size=0.2)
X_tr.shape,X_ts.shape

## 4. 建立逻辑回归模型

In [ ]:
from sklearn.linear_model import LogisticRegression
model_original = LogisticRegression()
model_original.fit(X_tr,y_tr.values.ravel())

In [ ]:
model_rs = LogisticRegression(class_weight='balanced')
model_rs.fit(X_tr,y_tr.values.ravel())

In [ ]:
model_roc = LogisticRegression()
model_roc.fit(X_tr,y_tr.values.ravel())

In [ ]:
from sklearn.metrics import roc_curve
import numpy as np
preds = model_roc.predict_proba(X_ts)[:,1]
fpr, tpr, thresh = roc_curve(y_ts, preds)
df_roc = pd.DataFrame(dict(fpr=fpr, tpr=tpr))
# 计算ROC曲线最优阈值
idx = np.argmax(tpr-fpr)
Thresh = thresh[idx]
# 绘制ROC曲线，查看阈值
plt.plot(fpr,tpr)
plt.scatter(fpr[idx],tpr[idx],c='r')
plt.text(fpr[idx],tpr[idx],'(%.4f, %.4f, %.4f)'%(fpr[idx],tpr[idx],Thresh),va='top')
plt.title('ROC curve')
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.grid(True)
plt.show()

## 5. 模型预测与评估

In [ ]:
# 普通逻辑回归模型预测与混淆矩阵
y_original_pred = model_original.predict(X_ts)
from sklearn.metrics import confusion_matrix,classification_report
confusion_matrix(y_ts,y_original_pred)

In [ ]:
# 重抽样方法预测与混淆矩阵
y_rs_pred = model_rs.predict(X_ts)
confusion_matrix(y_ts,y_rs_pred)

In [ ]:
# ROC阈值预测与混淆矩阵
pred_prob = model_roc.predict_proba(X_ts)
pred_thresh = [int(x[1]>Thresh) for x in pred_prob]
confusion_matrix(y_ts, pred_thresh)

## 6. 实验结论
1. 对于信用欺诈分析而言，业务目的是尽可能准确的识别欺诈用户，为此可以一定程度上“忍受”对非欺诈客户的“误伤”。
2. 未经处理的数据集模型，识别出63名欺诈用户，放过32名欺诈用户，误伤9名非欺诈用户。
3. 经过重抽样的模型，识别出91名欺诈用户，放过4名欺诈用户，误伤1362名非欺诈用户。
4. 使用ROC的模型，识别出90名欺诈用户，放过5名欺诈用户，误伤1623名非欺诈用户。
5. 本次分析中，原始的逻辑回归模型未能有效识别出欺诈用户，重抽样方法和ROC方法都比较准确的识别出了欺诈用户。
6. 本次分析中，重抽样方法略优于ROC方法。